In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import codecs
import datetime as dt
import time
import os
import os.path
import shutil


PATH_DIR_DOWNLOADS = os.path.join(f"c:\\Users\\{os.getlogin( )}\\Downloads")
PATH_DIR_DATA_STORE = os.path.join(f".\\Data_store")

In [3]:
driver = webdriver.Chrome()
driver.get('https://email.volganet.ru/?Skin=Samoware')

# Select the Login box
id_box = driver.find_element(By.NAME, 'username')
# Send Login box information
id_box.send_keys('d_korostelev')

# Find password box
pass_box = driver.find_element(By.NAME, 'Password')
# Send password box information
pass_box.send_keys('Ln##9uFT')
time.sleep(2)

# Find login button and Click 
login_button = driver.find_element(By.NAME, 'login').click()
time.sleep(2)

driver.find_element(By.XPATH, "//*[@id=\"samoware-mail\"]/div[1]/div[5]/ul/li[2]/span/span[1]").click()

find_messages_all = driver.find_elements(By.XPATH, "//*[@class=\"samoware-mail-list__item ng-scope\"]/div[1]/div")
find_messages_ods = [item for item in find_messages_all if "ОДС Главного управления МЧС России по Волгоградской области"in item.text]
time.sleep(1.5)

iframe = driver.find_element(By.XPATH, '//*[@id="samoware-mail"]/div[2]/div[3]/div[4]/div[1]/iframe')
list_download_files = list() 

for item in find_messages_ods:
    driver.find_element(By.XPATH, "//*[@id=\"samoware-mail\"]/div[1]/div[5]/ul/li[2]/span/span[1]").click()
    item.click()
    time.sleep(2)

    driver.switch_to.frame(iframe)

    time_start_download = time.time()
    files_find_messages = driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div/button[1]")
    files_find_messages.click()
 
    time.sleep(4)
    time_final_download = time.time()
    driver.switch_to.default_content()
            
    all_download_files = [[filename, os.stat(os.path.join(PATH_DIR_DOWNLOADS, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DOWNLOADS)]
    download_file = [file for file in all_download_files if file[1] >= time_start_download and  file[1] <= time_final_download]
    list_download_files.extend(download_file)
    
# print(dir(driver.switch_to))
# print(files_find_messages[0].tag_name)
# print(files_find_messages[0].text)
# print(files_find_messages[0].get_attribute("class"))
time.sleep(5)

## Проверка файлов и добавление

In [4]:
# Список всех файлов с временем последней модификации:
# Каждый элемент — это имя файла и время 
all_data_store_files = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE)]
all_data_store_files = sorted(all_data_store_files,  reverse=False, key=lambda x:x[1])

# Список всех файлов с временем последней модификации:
# список файлов и время представлены отдельными списками в списке
list_all_data_store_files = [[item[0] for item in all_data_store_files]]
list_all_data_store_files.append([item[1] for item in all_data_store_files])

for file_download in list_download_files:
    ful_path_download_file = os.path.join(PATH_DIR_DOWNLOADS, file_download[0])
    ful_path_data_store_file =  os.path.join(PATH_DIR_DATA_STORE, file_download[0])
    
    if file_download[0] not in list_all_data_store_files[0]:
        shutil.copyfile(ful_path_download_file, ful_path_data_store_file)
        print(f"File: {ful_path_download_file} added to {PATH_DIR_DATA_STORE}")
        print(1)
    elif file_download[0] in list_all_data_store_files[0] and file_download[1] > list_all_data_store_files[1][list_all_data_store_files[0].index(file_download[0])]:
        shutil.copyfile(os.path.join(PATH_DIR_DOWNLOADS, file_download[0]), os.path.join(PATH_DIR_DATA_STORE, file_download[0]))
        print(f"File: {ful_path_download_file} added to {PATH_DIR_DATA_STORE}")
        print(2)
    else:
        print(f"File: {ful_path_download_file} don't add to {PATH_DIR_DATA_STORE}")

File: c:\Users\AdminOS\Downloads\Долгосрочный прогноз Волгоградской области на 2024 утв.pdf added to .\Data_store
1
File: c:\Users\AdminOS\Downloads\3969-20-3-6 Прогноз ЧС по ЮФО на 08 декабря.pdf added to .\Data_store
1
File: c:\Users\AdminOS\Downloads\ИВ-219-22-545 Прогноз ЧС ВО на 08.12.2023.PDF added to .\Data_store
1
File: c:\Users\AdminOS\Downloads\2.Прогноз ВО недельный 07-13.12.2023 ИВ-219-4421.pdf added to .\Data_store
2
File: c:\Users\AdminOS\Downloads\ИВ-219-22-544 Прогноз ЧС ВО на 07.12.2023 (1).PDF added to .\Data_store
1


# Создание html страницы

In [5]:
pattern_file_name_CHS_UFO = 'Прогноз ЧС по ЮФО'
pattern_file_name_CHS_VO = 'Прогноз ЧС ВО'
pattern_file_name_KNP_VO = 'Прогноз ВО недельный'
pattern_file_name_4MOD = '4МОД'
pattern_file_name_IDIV = 'Информационное донесение'

all_files_name_CHS_UFO = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_CHS_UFO.lower() in filename.lower()]
all_files_name_CHS_UFO = sorted(all_files_name_CHS_UFO,  reverse=False, key=lambda x:x[1])
last_file_name_UFO = max(all_files_name_CHS_UFO)[0]

all_file_name_CHS_VO = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_CHS_VO.lower() in filename.lower()]
all_file_name_CHS_VO = sorted(all_file_name_CHS_VO,  reverse=False, key=lambda x:x[1])
last_file_name_CHS_VO = max(all_file_name_CHS_VO)[0]

all_file_name_KNP_VO = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_KNP_VO.lower() in filename.lower()]
all_file_name_KNP_VO = sorted(all_file_name_KNP_VO,  reverse=False, key=lambda x:x[1])
last_file_name_KNP_VO = max(all_file_name_KNP_VO)[0]

all_file_name_4MOD = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_4MOD.lower() in filename.lower()]
all_file_name_4MOD = sorted(all_file_name_4MOD,  reverse=False, key=lambda x:x[1])
last_file_name_4MOD = max(all_file_name_4MOD)[0]

all_file_name_IDIV = [[filename, os.stat(os.path.join(PATH_DIR_DATA_STORE, filename)).st_mtime] for filename in os.listdir(PATH_DIR_DATA_STORE) if pattern_file_name_IDIV.lower() in filename.lower()]
all_file_name_IDIV = sorted(all_file_name_IDIV,  reverse=False, key=lambda x:x[1])
last_file_name_IDIV = max(all_file_name_IDIV)[0]


In [6]:
site_page_str = f"""<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <title></title>
    <link rel="stylesheet" href="style.css">
</head>

<body>
    <div class="dashboard">
        <h1> ОДС Главного управления МЧС России по Волгоградской области</h1>
        <div class="container">
            <div class="block_nav_pdf">
                <div>
                    <h1>Мониторинг пожароопасной ситуации</h1>
                    <p>{last_file_name_4MOD}</p>
                    <input id="color-input1" type="button" value="Показать" onmousedown="viewDiv1()">
                </div>
                <div>
                    <h1>Оперативный ежедневный сводный прогноз ЮФО</h1>
                    <p>{last_file_name_UFO}</p>
                    <input id="color-input2" type="button" value="Показать" onmousedown="viewDiv2()">
                </div>
                <div>
                    <h1>Оперативный ежедневный сводный прогноз ВО</h1>
                    <p>{last_file_name_CHS_VO}</p>
                    <input id="color-input3" type="button" value="Показать" onmousedown="viewDiv3()">
                </div>
                <div>
                    <h1>Краткосрочный недельный прогноз ВО</h1>
                    <p>{last_file_name_KNP_VO}</p>
                    <input id="color-input4" type="button" value="Показать" onmousedown="viewDiv4()">
                </div>
                <div>
                    <h1>Информационное донесение</h1>
                    <p>{last_file_name_IDIV}</p>
                    <input id="color-input5" type="button" value="Показать" onmousedown="viewDiv5()">
                </div>
            </div>

            <div class="block_view_pdf">
                <div id="MOD">
                    <embed src="Data_store/{last_file_name_4MOD}" />
                </div>
                <div id="CHS_UFO">
                    <embed src="Data_store/{last_file_name_UFO}" />
                </div>
                <div id="CHS_VO">
                    <embed src="Data_store/{last_file_name_CHS_VO}" />
                </div>
                <div id="KNP_VO">
                    <embed src="Data_store/{last_file_name_KNP_VO}" />
                </div>
                <div id="IDIV">
                    <embed src="Data_store/{last_file_name_IDIV}" />
                </div>
            </div>
        </div>

    </div>
    <script>
        function viewDiv1() {{
            document.getElementById("MOD").style.display = "block";
            document.getElementById("CHS_UFO").style.display = "none";
            document.getElementById("CHS_VO").style.display = "none";
            document.getElementById("KNP_VO").style.display = "none";
            document.getElementById("IDIV").style.display = "none";

            document.getElementById("color-input1").style.color = "#ffffff"
            document.getElementById("color-input2").style.color = "#000"
            document.getElementById("color-input3").style.color = "#000"
            document.getElementById("color-input4").style.color = "#000"
            document.getElementById("color-input5").style.color = "#000"
            document.getElementById("color-input1").style.background = "rgb(49, 70, 139)";
            document.getElementById("color-input2").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input3").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input4").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input5").style.background = "hsl(225, 19%, 92%)";
        }};
        function viewDiv2() {{
            document.getElementById("MOD").style.display = "none";
            document.getElementById("CHS_UFO").style.display = "block";
            document.getElementById("CHS_VO").style.display = "none";
            document.getElementById("KNP_VO").style.display = "none";
            document.getElementById("IDIV").style.display = "none";
            document.getElementById("color-input1").style.color = "#000"
            document.getElementById("color-input2").style.color = "#ffffff"
            document.getElementById("color-input3").style.color = "#000"
            document.getElementById("color-input4").style.color = "#000"
            document.getElementById("color-input5").style.color = "#000"
            document.getElementById("color-input1").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input2").style.background = "rgb(49, 70, 139)";
            document.getElementById("color-input3").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input4").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input5").style.background = "hsl(225, 19%, 92%)";
        }};
        function viewDiv3() {{
            document.getElementById("MOD").style.display = "none";
            document.getElementById("CHS_UFO").style.display = "none";
            document.getElementById("CHS_VO").style.display = "block";
            document.getElementById("KNP_VO").style.display = "none";
            document.getElementById("IDIV").style.display = "none";
            document.getElementById("color-input1").style.color = "#000"
            document.getElementById("color-input2").style.color = "#000"
            document.getElementById("color-input3").style.color = "#ffffff"
            document.getElementById("color-input4").style.color = "#000"
            document.getElementById("color-input5").style.color = "#000"
            document.getElementById("color-input1").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input2").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input3").style.background = "rgb(49, 70, 139)";
            document.getElementById("color-input4").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input5").style.background = "hsl(225, 19%, 92%)";
        }};

        function viewDiv4() {{
            document.getElementById("MOD").style.display = "none";
            document.getElementById("CHS_UFO").style.display = "none";
            document.getElementById("CHS_VO").style.display = "none";
            document.getElementById("KNP_VO").style.display = "block";
            document.getElementById("IDIV").style.display = "none";

            document.getElementById("color-input1").style.color = "#000"
            document.getElementById("color-input2").style.color = "#000"
            document.getElementById("color-input3").style.color = "#000"
            document.getElementById("color-input4").style.color = "#ffffff"
            document.getElementById("color-input5").style.color = "#000"
            document.getElementById("color-input1").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input2").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input3").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input4").style.background = "rgb(49, 70, 139)";
            document.getElementById("color-input5").style.background = "hsl(225, 19%, 92%)";
        }};

        function viewDiv5() {{
            document.getElementById("MOD").style.display = "none";
            document.getElementById("CHS_UFO").style.display = "none";
            document.getElementById("CHS_VO").style.display = "none";
            document.getElementById("KNP_VO").style.display = "none";
            document.getElementById("IDIV").style.display = "block";

            document.getElementById("color-input1").style.color = "#000"
            document.getElementById("color-input2").style.color = "#000"
            document.getElementById("color-input3").style.color = "#000"
            document.getElementById("color-input4").style.color = "#000"
            document.getElementById("color-input5").style.color = "#ffffff"
            document.getElementById("color-input1").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input2").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input3").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input4").style.background = "hsl(225, 19%, 92%)";
            document.getElementById("color-input5").style.background = "rgb(49, 70, 139)";
        }};

        function myFunctionSearch() {{
            var input = document.getElementById("Search");
            var filter = input.value.toLowerCase();
            var nodes = document.getElementsByClassName('camera_block');

            for (i = 0; i < nodes.length; i++) {{
                if (nodes[i].innerText.toLowerCase().includes(filter)) {{
                    nodes[i].style.display = "block";
                }} else {{
                    nodes[i].style.display = "none";
                }}
            }}
        }}
    </script>
</body>

</html>"""

# Html_file = codecs.open("index — копия.html","w", "utf-8")
Html_file = codecs.open("index.htm", "w", "utf-8")
Html_file.write(site_page_str)
Html_file.close()

# files_to_site_list = [f'''<tr>
#                 <td width="20px">{len(all_data_store_files) - i}.</td>
#                 <td>{all_data_store_files[i][0]}</td>
#                 <td><a
#                     href="Data_store/{all_data_store_files[i][0]}" target="_blank" download="{all_data_store_files[i][0]}">
#                     <img src="image/icon_dowload.png" alt="download" width="20px" height="20px"></a></td>
#             </tr>''' for i in range(len(all_data_store_files))]
# files_to_site_str = ("\n").join(files_to_site_list)


# site_page_str = f"""<!DOCTYPE html>
# <html>

# <head>
#     <meta charset="utf-8">
#     <title></title>
#     <link rel="stylesheet" href="style.css">
# </head>

# <body>
#     <div class="dashboard">
#         <h1> ОДС Главного управления МЧС России по Волгоградской области</h1>
#         <div class="container">
#         <table>
#             {files_to_site_str}
#         </table>
#     </div>

#     </div>
# </body>

# </html>
# """